In [3]:
import pickle
import sncosmo
import numpy as np
import lsst.daf.base as dafBase
import lsst.daf.persistence as dafPersist

import lsst.afw.display.ds9 as ds9
import lsst.afw.image as afwImage
import lsst.afw.display.utils as displayUtils

In [5]:
i=1
DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_master/"
butler = dafPersist.Butler(DATADIR+"/output")



im = butler.get('calexp', {'visit': 836493 , 'filter':'r' , 'ccd':0})
    
placeholder = im.Factory(im)

placeholder = afwImage.ExposureF(placeholder.getWidth(), placeholder.getHeight())


In [6]:
ds9.mtv(placeholder, frame=1)

ds9 doesn't appear to be running (XPA: XPA returned NULL, (;iconify no; raise)), I'll try to exec it for you


In [9]:
mosaic = displayUtils.Mosaic(gutter=50, background=50, mode="y")
mosaic_line = None

for i in range(0,36):

    
    if i%9 == 0:
        #print i
        if mosaic_line is not None:
            ml = mosaic_line.makeMosaic(frame=None, display=None)
            mosaic.append(ml)
            
        mosaic_line = displayUtils.Mosaic(gutter=50, background=50, mode='x')
        
    if butler.datasetExists('deepDiff_differenceExp', {'visit': 836493 , 'filter':'r' , 'ccd':i}):
        cl = butler.get('deepDiff_differenceExp', {'visit': 836493 , 'filter':'r' , 'ccd':i})
    else:
        cl = placeholder
         
    #print cl
        
    mosaic_line.append(cl.getMaskedImage().getImage(), label=str(i))
        
ml = mosaic_line.makeMosaic(frame=None, display=None)
mosaic.append(ml)  

    
mosaic.makeMosaic(frame=10)



<lsst.afw.image.imageLib.ImageF; proxy of <Swig Object of type 'boost::shared_ptr< lsst::afw::image::Image< float > > *' at 0x7fce8be808d0> >

In [10]:
cl = butler.get('deepDiff_differenceExp', {'visit': 836493 , 'filter':'r' , 'ccd':14})
ds= butler.get('deepDiff_diaSrc', {'visit': 836493 , 'filter':'r' , 'ccd':14})


In [11]:
ds9.mtv(cl, frame=2)

In [14]:
for s in ds:
    ds9.dot('o',s.getX(), s.getY(), frame=2, size=10)

In [15]:
import DiaSourceTools as DSTools

In [16]:
fluxes, sigmas = DSTools.get_fluxes_and_sigmas(ds, "base_PeakLikelihoodFlux_flux")

m_sigma = np.mean(sigmas)
t_sigma = 5*m_sigma

sources = DSTools.get_sources_over_sigma(ds, t_sigma, "base_PeakLikelihoodFlux_flux")

In [20]:
for s in sources:
    ds9.dot('o',s.getX(), s.getY(), frame=2, size=25, ctype=ds9.RED)